#  softvoting
* models
    - vit
    - efficient
    - caformer
- kfold
* result
    - public 점수 : 0.8619097833
    - private 점수 : 0.864669898

In [11]:
import gc
import os
import random
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import timm

import warnings
warnings.filterwarnings(action='ignore') 

In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [13]:
CFG = {
    'IMG_SIZE_VIT': 260,
    'IMG_SIZE_EFFICIENT': 300,
    'IMG_SIZE_CAFORMER': 272,
    'BATCH_SIZE': 16,
    'K-FOLD': 5,
    'FILENAME': 'soft',
    'SEED': 6
}

In [14]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [15]:
running_colab = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
if running_colab:
    from google.colab import drive
    drive.mount('/content/drive')
if running_colab:
    data_path = '/content/drive/MyDrive/Colab Notebooks/ai6th/data/optiver/'
else:
    data_path = '../../data/'

In [16]:
df = pd.read_csv(os.path.join(data_path, 'train.csv'))
df.loc[3896, 'artist'] = 'Titian'
df.loc[3986, 'artist'] = 'Alfred Sisley'
df.head()

,id,img_path,artist
0,0,./train/0000.jpg,Diego Velazquez
1,1,./train/0001.jpg,Vincent van Gogh
2,2,./train/0002.jpg,Claude Monet
3,3,./train/0003.jpg,Edgar Degas
4,4,./train/0004.jpg,Hieronymus Bosch


In [17]:
# Label Encoding
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)

In [18]:
def get_data(df, infer=False):
    if infer:
        return df['img_path'].apply(lambda p: os.path.join(data_path, p)).values
    return df['img_path'].apply(lambda p: os.path.join(data_path, p)).values, df['artist'].values

In [19]:
from torchvision.transforms import ToTensor


class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms if transforms else ToTensor()

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']
        
        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_paths)

In [20]:
train_transform_vit = A.Compose([
    A.Resize(CFG['IMG_SIZE_VIT']*2,CFG['IMG_SIZE_VIT']*2),
    A.RandomCrop(CFG['IMG_SIZE_VIT'],CFG['IMG_SIZE_VIT']),
    A.Transpose(p=0.5), # 행렬 스왑
    A.HorizontalFlip(p=0.5), # 좌우 반전
    A.VerticalFlip(p=0.5), # 상하 반전
    A.ShiftScaleRotate(p=0.5), # 랜덤하게 옮기고, scale, 회전
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.5), # 빛깔, 색조, 값 변환
    A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5), # 명도 대비
    A.ChannelShuffle(), # RGB 채널 간 shuffle
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

test_transform_vit = A.Compose([
    A.Resize(CFG['IMG_SIZE_VIT'],CFG['IMG_SIZE_VIT']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

In [21]:
train_transform_efficient = A.Compose([
    A.Resize(CFG['IMG_SIZE_EFFICIENT']*2,CFG['IMG_SIZE_EFFICIENT']*2),
    A.RandomCrop(CFG['IMG_SIZE_EFFICIENT'],CFG['IMG_SIZE_EFFICIENT']),
    A.Transpose(p=0.5), # 행렬 스왑
    A.HorizontalFlip(p=0.5), # 좌우 반전
    A.VerticalFlip(p=0.5), # 상하 반전
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    A.CoarseDropout(p=0.5),
    ToTensorV2()
])

test_transform_efficient = A.Compose([
    A.Resize(CFG['IMG_SIZE_EFFICIENT'],CFG['IMG_SIZE_EFFICIENT']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

In [22]:
train_transform_caformer = A.Compose([
    A.Resize(CFG['IMG_SIZE_CAFORMER']*2,CFG['IMG_SIZE_CAFORMER']*2),
    A.RandomCrop(CFG['IMG_SIZE_CAFORMER'],CFG['IMG_SIZE_CAFORMER']),
    A.Transpose(p=0.5), # 행렬 스왑
    A.HorizontalFlip(p=0.5), # 좌우 반전
    A.VerticalFlip(p=0.5), # 상하 반전
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    A.CoarseDropout(p=0.5),
    ToTensorV2()
])

test_transform_caformer = A.Compose([
    A.Resize(CFG['IMG_SIZE_CAFORMER'],CFG['IMG_SIZE_CAFORMER']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

In [23]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(0)

In [24]:
class VitModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(VitModel, self).__init__()
        self.backbone = timm.create_model('tiny_vit_21m_384.dist_in22k_ft_in1k', pretrained=True, num_classes=0)
        self.classifier = nn.Sequential(
            nn.LayerNorm(576),
            nn.GELU(),
            nn.Dropout(p=0.4),
            nn.Linear(576, num_classes)
        )
            
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [25]:
class EfficientNetModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(EfficientNetModel, self).__init__()
        self.backbone = timm.create_model('efficientnet_b3', pretrained=True, num_classes=512)
        self.classifier = nn.Sequential(
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(p=0.4),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [26]:
class CAFormerModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(CAFormerModel, self).__init__()
        self.backbone = timm.create_model('caformer_s18.sail_in22k_ft_in1k_384', pretrained=True, num_classes=0)
        self.classifier = nn.Sequential(
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(p=0.4),
            nn.Linear(512, num_classes)
        )
            
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [27]:
def clear_mem():
    gc.collect()
    torch.cuda.empty_cache()

In [28]:
time_now = datetime.now()
print(f'{time_now=}')

time_now=datetime.datetime(2023, 12, 28, 10, 23, 5, 441516)


In [29]:
test_df = pd.read_csv(os.path.join(data_path, './test.csv'))
test_df.head()

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg


In [30]:
test_img_paths = get_data(test_df, infer=True)

In [31]:
test_dataset_vit = CustomDataset(test_img_paths, None, test_transform_vit)
test_loader_vit = DataLoader(test_dataset_vit, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [32]:
test_dataset_efficient = CustomDataset(test_img_paths, None, test_transform_efficient)
test_loader_efficient = DataLoader(test_dataset_efficient, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [33]:
test_dataset_caformer = CustomDataset(test_img_paths, None, test_transform_caformer)
test_loader_caformer = DataLoader(test_dataset_caformer, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [34]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for idx, img in enumerate(test_loader):
            img = img.float().to(device)
            
            model_pred = model(img).detach().cpu()
            model_pred = F.softmax(model_pred, dim=1)
            model_preds.extend(model_pred.numpy().tolist())
    
    print('Done.')
    return model_preds

In [35]:
preds_kfold = np.zeros((len(test_df), len(le.classes_)))
test_targets = [
    (test_loader_vit, 'kfold_vit'),
    (test_loader_efficient, 'kfold_efficientnet'),
    (test_loader_caformer, 'kfold_caformer')
]
for loader, run_id in test_targets:
    for k_ in range(CFG['K-FOLD']):
        checkpoint = os.path.join(data_path, f'runs/{run_id}/best_model_{k_}.pt')
        print(f'{k_}-fold CHECKPOINT LOADED: {checkpoint}')
        infer_model = torch.load(checkpoint)
        ret = inference(infer_model, loader, device)
        preds_kfold += np.array(ret)

0-fold CHECKPOINT LOADED: ../../data/runs/kfold_vit/best_model_0.pt
Done.
1-fold CHECKPOINT LOADED: ../../data/runs/kfold_vit/best_model_1.pt
Done.
2-fold CHECKPOINT LOADED: ../../data/runs/kfold_vit/best_model_2.pt
Done.
3-fold CHECKPOINT LOADED: ../../data/runs/kfold_vit/best_model_3.pt
Done.
4-fold CHECKPOINT LOADED: ../../data/runs/kfold_vit/best_model_4.pt
Done.
0-fold CHECKPOINT LOADED: ../../data/runs/kfold_efficientnet/best_model_0.pt
Done.
1-fold CHECKPOINT LOADED: ../../data/runs/kfold_efficientnet/best_model_1.pt
Done.
2-fold CHECKPOINT LOADED: ../../data/runs/kfold_efficientnet/best_model_2.pt
Done.
3-fold CHECKPOINT LOADED: ../../data/runs/kfold_efficientnet/best_model_3.pt
Done.
4-fold CHECKPOINT LOADED: ../../data/runs/kfold_efficientnet/best_model_4.pt
Done.
0-fold CHECKPOINT LOADED: ../../data/runs/kfold_caformer/best_model_0.pt
Done.
1-fold CHECKPOINT LOADED: ../../data/runs/kfold_caformer/best_model_1.pt
Done.
2-fold CHECKPOINT LOADED: ../../data/runs/kfold_caformer/

In [36]:
preds_kfold[:10]

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.005496,0.027346,0.012067,0.014264,0.004366,0.008614,0.011494,0.015468,0.002160,0.078608,...,0.055010,0.031977,0.087596,0.017154,0.021281,0.055630,0.332400,0.008772,1.039908,0.009672
1,0.014726,0.014147,14.005913,0.006503,0.009794,0.004523,0.002861,0.007013,0.041538,0.011429,...,0.010602,0.006560,0.010462,0.051714,0.007542,0.007914,0.016960,0.004358,0.050175,0.006135
2,0.019637,0.016027,0.005989,0.006498,0.007108,0.004041,12.361973,0.004771,0.006017,0.209205,...,0.009100,0.193100,0.041416,0.013944,0.149779,0.023629,0.626744,0.005324,0.023706,0.006364
3,14.294950,0.017102,0.008565,0.009153,0.006390,0.005516,0.001851,0.010596,0.003374,0.004476,...,0.117263,0.003873,0.007805,0.017470,0.004143,0.006000,0.009181,0.004133,0.160420,0.007656
4,0.012188,0.120251,1.479323,0.015547,0.011573,0.042609,0.002000,0.054094,0.063222,0.009724,...,0.022550,0.003423,0.010805,0.019424,0.004506,0.007089,0.016069,0.009677,3.789705,0.004692


In [37]:
preds = preds_kfold.argmax(axis=1)

In [38]:
preds = le.inverse_transform(preds)

In [39]:
submit = pd.read_csv(os.path.join(data_path, './sample_submission.csv'))

In [40]:
submit['artist'] = preds

In [41]:
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Caravaggio
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Pablo Picasso


In [42]:
submit.to_csv(os.path.join(data_path, f"./submit_{CFG['FILENAME']}.csv"), index=False)